<a href="https://colab.research.google.com/github/javierfuenca/clasificador_verificabilidad/blob/main/clasificador_verificabilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea Newtral
Javier de la Fuente Casal

## 1. Obtención y preprocesado de datos

Descargamos el dataset de una URL de la forma:


```
https://bucket-name.s3.Region.amazonaws.com/key
```

### 1.1 Descarga de datos

In [1]:
# descargamos fichero
!wget https://ml-coding-test.s3.eu-west-1.amazonaws.com/claims.bson

--2020-10-19 14:53:45--  https://ml-coding-test.s3.eu-west-1.amazonaws.com/claims.bson
Resolving ml-coding-test.s3.eu-west-1.amazonaws.com (ml-coding-test.s3.eu-west-1.amazonaws.com)... 52.218.21.171
Connecting to ml-coding-test.s3.eu-west-1.amazonaws.com (ml-coding-test.s3.eu-west-1.amazonaws.com)|52.218.21.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5735425 (5.5M) [binary/octet-stream]
Saving to: ‘claims.bson’

claims.bson         100%[===================>]   5.47M  4.99MB/s    in 1.1s    

2020-10-19 14:53:47 (4.99 MB/s) - ‘claims.bson’ saved [5735425/5735425]



In [2]:
import bson
with open('claims.bson', 'rb') as file:
    b = bson.decode_all(file.read())

In [3]:
print(f"Datos de tipo: {type(b)}")
print(f"Tamaño del dataset: {len(b)}")

Datos de tipo: <class 'list'>
Tamaño del dataset: 14353


### 1.2 Preprocesado

Lectura y adaptación a pandas dataframe para mejor tratamiento de los datos

In [4]:
import pandas as pd

df = pd.DataFrame(b)
df = df.set_index("_id")
df.head()

,text_es,text_en,text_fr,claim
_id,,,,
5f80940cf95f926ca81a3751,Gracias.,Thank you.,Merci.,0
5f80940cf95f926ca81a3752,"Por ejemplo, cuando estamos hablando de un pa...","For example, when we are talking about a coun...","Par exemple, quand on parle d’un pays qui doi...",0
5f80940cf95f926ca81a3753,Entonces como solo creo que es como la políti...,So as I just think it's like the politics of ...,"Donc, comme je pense que c’est comme la polit...",0
5f80940cf95f926ca81a3754,Y pienso que el Partido Popular no ha estado ...,And I think the People's Party has not risen ...,Et je pense que le Parti populaire n’a pas ét...,0
5f80940cf95f926ca81a3755,Lo siguiente Nos vamos ya volando y les dejo ...,The next thing We fly and I leave you with th...,La prochaine chose que nous volons et je vous...,0


In [5]:
df.describe()

,claim
count,14353.000000
mean,0.074201
std,0.262106
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Hay un claro desbalanceo entre frases verificables (claim==1) y no verificables (claim==0)

In [6]:
print(f'Hay un total de {df["claim"][df["claim"]==1].count()} de frases verificables')
print(f'Hay un total de {df["claim"][df["claim"]==0].count()} de frases no verificables')

Hay un total de 1065 de frases verificables
Hay un total de 13288 de frases no verificables


Escogemos unas muestras aleatorias para observar los ejemplos de frases verificables y no verificables

In [7]:
import numpy as np
A = df[df["claim"]==1].values
print(A[np.random.choice(A.shape[0])])

B = df[df["claim"]==0].values
print(B[np.random.choice(B.shape[0])])

[' Hay otra cuestión adicional y es que mucha gente está recibiendo estos ertes y va a tener dos pagadores su empresa y el Servicio Público de Empleo.'
 " There's another additional issue and it's that a lot of people are getting these ertes and they're going to have two payers their company and the Public Employment Service."
 ' Il y a un autre problème et c’est que beaucoup de gens obtiennent ces ertes et qu’ils vont avoir deux payeurs dans leur entreprise et le Service public de l’emploi.'
 1]
[' Es precisamente lo que está ocurriendo estos últimos días en Lérida con el presidente de la Generalitat.'
 ' This is precisely what is happening in lleida these last days with the President of the Generalitat.'
 ' C’est précisément ce qui se passe à Lleida ces derniers jours avec le Président de la Generalitat.'
 0]


### 1.3 Balanceo de datos

Para alcanzar un equilibrio entre la cantidad de frases verificables y no verificables y evitar sesgos en la red neuronal truncaremos una muestra de frases no verificables para igualar en tamaño a las verificables. Separamos usando random state para obtener el mismo resultado en todas las ejecuciones

In [8]:
new_df = pd.concat([df[df["claim"]==0].sample(df["claim"][df["claim"]==1].count(), random_state=2020), df[df["claim"]==1]])

In [9]:
print(f'Hay un total de {new_df["claim"][new_df["claim"]==1].count()} de frases verificables')
print(f'Hay un total de {new_df["claim"][new_df["claim"]==0].count()} de frases no verificables')

Hay un total de 1065 de frases verificables
Hay un total de 1065 de frases no verificables


In [10]:
# instalamos ktrain en Google Colab para una sencilla implementación de BERT
!pip3 install ktrain

In [11]:
# importamos ktrain y ktrain.text
import ktrain
from ktrain import text

ktrain.__version__

'0.23.0'

Con ayuda de pandas desdoblamos las frases en los 3 idiomas para triplicar la cantidad de datos

In [12]:
df_comb = new_df.copy()
df_comb["combined"] = df_comb[["text_es", "text_en", "text_fr"]].values.tolist()
df_comb = df_comb.drop(columns=["text_es", "text_en", "text_fr"])
df_comb = df_comb.explode("combined")

In [13]:
print(f'Hay un total de {df_comb["claim"][df_comb["claim"]==1].count()} de frases verificables')
print(f'Hay un total de {df_comb["claim"][df_comb["claim"]==0].count()} de frases no verificables')

Hay un total de 3195 de frases verificables
Hay un total de 3195 de frases no verificables


De todo nuestro dataset ya balanceado y listo para procesar separamos un 80% para entrenamiento y un 20% para test. Al igual que en el paso anterior, escogemos un random_state arbitrario para obtener una misma ejecución

In [14]:
from sklearn.model_selection import train_test_split
X, y = df_comb["combined"].values, df_comb["claim"].values

X_train_data, X_test_data, y_train_data, y_test_data = train_test_split(X, y, test_size=0.2, random_state=2020)

## 2. Preparación y entrenamiento del modelo


Procesamos con la ayuda de ktrain los datos de entrenamiento y test para adaptarlo a BERT multilenguas


In [15]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train_data, y_train=y_train_data,
                                                                       x_test=X_test_data, y_test=y_test_data,
                                                                       class_names=['no_verificable', 'verificable'],
                                                                       preprocess_mode='bert',
                                                                       lang='multi',
                                                                       maxlen=350,
                                                                       max_features=35000)

preprocessing train...
language: multi


Is Multi-Label? False
preprocessing test...
language: multi


task: text classification


Cargamos el modelo de tipo BERT

In [16]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


Calculamos tasa de aprendizaje óptima

In [17]:
#learner.lr_find(max_epochs=5)

In [18]:
#learner.lr_plot()

In [19]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
852/852 [==============================] - 1085s 1s/step - loss: 0.3705 - accuracy: 0.8255


## 3. Validación y pruebas

### 3.1 Pruebas sobre conjunto de test

In [20]:
learner.validate(val_data=(x_test, y_test), class_names=['no_verificable', 'verificable'])

                precision    recall  f1-score   support

no_verificable       0.91      0.90      0.91       620
   verificable       0.91      0.92      0.91       658

      accuracy                           0.91      1278
     macro avg       0.91      0.91      0.91      1278
  weighted avg       0.91      0.91      0.91      1278



array([[559,  61],
       [ 54, 604]])

In [21]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [22]:
predictor.get_classes()

['no_verificable', 'verificable']

Ejemplo sobre una muestra aleatoria

In [24]:
idx=np.random.randint(0, len(X_test_data)-1)
print(X_test_data[idx],"\nGT:\t", predictor.get_classes()[y_test_data[idx]], "\nPred:\t", predictor.predict(X_test_data[idx]))

 Nous précisons clairement que la conditionnalité de ces ressources qui sont des transferts et non des prêts, contrairement à la crise précédente, du moins dans une moindre mesure dans la moitié des ressources, est pratiquement conditionnelle à utiliser. 
GT:	 verificable 
Pred:	 verificable


### 3.2 Pruebas sobre frases no verificables descartadas

In [25]:
df_test = pd.concat([df, new_df, new_df]).drop_duplicates(keep=False)

In [26]:
contador = 0
for dato in df_test["text_es"].values:
    if predictor.predict(dato) == 'verificable':
        contador += 1

In [27]:
print(f"Total {contador}/{len(df_test['text_es'].values)}")

Total 1437/11710


### 3.3 Resultados en CSV

In [39]:
# Dataframe auxiliar para tener todo ordenado
results_df = pd.DataFrame(columns=["Frase", "Groundtruth", "Predicción"])
for i, phrase in enumerate(X_test_data):
    pred = np.argmax(predictor.predict_proba(phrase))
    results_df = results_df.append({"Frase": phrase, "Groundtruth": y_test_data[i], "Predicción": pred}, ignore_index=True)

In [40]:
results_df.to_csv("results.csv", index=False)